In [1]:
#This code uses a best feature select of 12 and calculates the results for 
# applying a moving average as well as 3 previous values.

from sklearn.datasets import fetch_openml
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn.linear_model as skl_lm
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score

from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures

from sklearn import utils
from sklearn import preprocessing
from sklearn import neighbors
from pandas import read_csv

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn import preprocessing

from sklearn.feature_selection import SelectKBest
from copy import deepcopy
import csv
lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()
kfld = KFold(n_splits=3,random_state=42)

In [2]:
#Reads data in from data folder.

data = read_csv('data/ethylene_methane.txt',delim_whitespace=True)
print(data.shape)

(4178504, 19)


In [3]:
#creates a new attribute thats the % Methane, in cases where we divide by zero
#we recieve a NAN which is replaced by a 0
#the 4th line drops that attributes that we defined to be unnecasry from our K
#best select we ran in a previous part of our code
#this code also finds all unique PPM for methane

data['Methane%'] = (data['Methane(ppm)']/(data['Methane(ppm)']+data['Ethylene(ppm)']))
data['Methane%'] = data['Methane%'].fillna(0)

ppm = data['Methane(ppm)'].unique()

data = data.drop(columns = ['Time(sec)', 'sensor1', 'sensor2','sensor9','sensor10'])

sensor_name = ['sensor3', 'sensor4', 'sensor5','sensor6', 'sensor7', 'sensor8', 'sensor11','sensor12', 'sensor13', 'sensor14', 'sensor15', 'sensor16']


In [4]:
#This block searches for Outliers for each PPM of Methane and then drops them
#from the file.

for j in sensor_name:
    print(j)
    for i in np.sort(ppm):    
        data2 = data[data["Methane(ppm)"] == i] #only select a specific PPM to begin trimming
        
        results = data2[j].quantile([0.125,0.875]) #Next lines calculate the range for non-Outliers
        IQR_15 = 1.5*(results.loc[0.875] - results.loc[0.125])
        fq = results.loc[0.125] - IQR_15
        tq = results.loc[0.875] + IQR_15
                
        index = data2[(fq>data2[j]) | (tq<data2[j])] #Finds Outliers
        data = data.drop(index.index) #Uses index of Outliers to drop rows from data 
        
        
        #print('Methane(ppm) = ',i)
        #print('Outlier range',fq , tq)
        #print(data.shape)
        #print("")

sensor3
sensor4
sensor5
sensor6
sensor7
sensor8
sensor11
sensor12
sensor13
sensor14
sensor15
sensor16


In [5]:
#Calculates a moving average of 6 and then adds 3 addition attributes each one
#belonging to previous value.

df = deepcopy(data)
for k in sensor_name:
    data[k+'avg6'] = df[k].rolling(window=6).mean()
    data[k+'(t-1)']=df[k].shift(1)
    data[k+'(t-2)']=df[k].shift(2)
    data[k+'(t-3)']=df[k].shift(3)
    print(k)
data = data.dropna()

sensor3
sensor4
sensor5
sensor6
sensor7
sensor8
sensor11
sensor12
sensor13
sensor14
sensor15
sensor16


In [6]:
#Defines our X and y and sets y equal to 1 if the Methane % is greater than 50%
#and splites the data into train and test

X_best = data.drop(columns = ['Methane(ppm)','Ethylene(ppm)','Methane%'])
y = np.where(data['Methane%'] >= 0.5, 1, 0)

start =time.time()
X_train, X_test, y_train, y_test = train_test_split(X_best, y, test_size=0.50, random_state=42) 
end = time.time()
print('Time (sec) = ',(end-start))

Time (sec) =  2.355138063430786


In [7]:
#these next blocks of code are the model fitting, each block attempts to fit 
#one model and records the start time starting at the fitting and ends after 
#finishing the CV
#The score and time printed once the code finishes

start =time.time()
clf = LogisticRegression(random_state=42, solver='lbfgs',max_iter=10000,n_jobs = -1).fit(X_train, y_train)
score = cross_val_score(clf, X_test, y_test, cv=kfld, scoring='r2').mean()
end = time.time()
print(score)
print('Time (sec) = ',(end-start))
#0.9203716676003069
#Time (sec) =  3024.461760520935

0.92022128452923
Time (sec) =  3587.8302414417267


In [8]:
start =time.time()
lda.fit(X_train, y_train)
score = cross_val_score(lda, X_test, y_test, cv=kfld, scoring='r2').mean()
end = time.time()
print(score)
print('Time (sec) = ',(end-start))
#0.9076945985630674 Time (sec) =  41.538795709609985

0.9076531773997442
Time (sec) =  48.227792501449585


In [9]:
start =time.time()
qda.fit(X_train, y_train)
score = cross_val_score(qda, X_test, y_test, cv=kfld, scoring='r2').mean()
end = time.time()
print(score)
print('Time (sec) = ',(end-start))
#0.8736068927766212 Time (sec) =  17.918303966522217

0.8688379480943459
Time (sec) =  19.58478307723999


In [10]:
start =time.time()
knn = neighbors.KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
score = cross_val_score(knn, X_test, y_test, cv=kfld, scoring='r2').mean()
end = time.time()
print(score)
print('Time (sec) = ',(end-start))

#0.9960584216483955 Time (sec) =  544.3553040027618

0.9846348814616667
Time (sec) =  1855.9880166053772


In [11]:
#these next lines take the previously fitted models and computes
#recall and precision

clf_y = clf.predict(X_test)
print(recall_score(y_test, clf_y))
print(precision_score(y_test, clf_y))

0.9573580128735591
0.9991923398335193


In [12]:
lda_y = lda.predict(X_test)
print(recall_score(y_test, lda_y))
print(precision_score(y_test, lda_y))

0.9529956297890042
0.9965713016073485


In [13]:
qda_y = qda.predict(X_test)
print(recall_score(y_test, qda_y))
print(precision_score(y_test, qda_y))

0.9579898371347161
0.9707253451542429


In [14]:
knn_y = knn.predict(X_test)
print(recall_score(y_test, knn_y))
print(precision_score(y_test, knn_y))

0.995277533000237
0.9965847874114144
